# Pre-AI Email Marketing - "Spray and Pray"

We've all received these form emails where it's clear that they just spit out the same email to every person in their email list, rather than actually looking at what kind of customer you are.

Sometimes it's even worse than just slapping your first name on the pre-amble, I'm sure you've received emails like this before too:

```
Dear $fname,

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras semper at urna sit amet imperdiet. Integer rutrum tempor tellus et porta. Mauris elit dui, euismod a ipsum sit amet, ultricies dictum ex. Suspendisse bibendum suscipit leo sed egestas. Suspendisse potenti. Maecenas placerat lectus eget tincidunt posuere. Ut aliquet elit ante. Duis vel volutpat massa, pulvinar condimentum enim. Aliquam erat volutpat. Sed vulputate pretium mauris, eu tempus arcu lacinia sed. Aliquam lobortis massa at ligula aliquet tincidunt. Aliquam ultrices sem a ultricies pulvinar. In facilisis pretium cursus. Vestibulum et malesuada justo. Vivamus erat ex, aliquet nec porttitor eu, sodales consectetur dolor.

Sincerely,
Marketing Person
@ Company
```

Below is a simplistic example of creating that email:

In [ ]:
people = ["Alice", "Bob", "Charlie", "David", "Emma", "Frank", "Grace", "Henry", "Ivy", "Jack"]

for person in people:
    message = f"Hey {person},\n Check out our web analytics platform, it's Awesome! It's perfect for your needs. Buy it now!\n - Marketer John"
    print(message)

# ChatGPT has entered the chat

If there's something that will help you stick out as a marketer, it's when you send emails that are roughly the same in content but you use ChatGPT to generate the actual content. Below you'll see how you could prompt OpenAI for some "unique" emails to send out to folks.

## Setup OpenAI

Below we'll install OpenAI's python module, enter in our API key (which will be terminated at the end of this webinar), define the list of people (defined above), and then the system prompt.

The system prompt helps prime ChatGPT to respond with content that is helpful for us. You'll notice that we explicity request for ChatGPT to lookup the top 5 web analytics companies in order to give it a small corpus of content to work from for this demo.

In [ ]:
!pip install openai --quiet

import openai

openai.api_key = ""

# Define the list of people

system = 'You are a helpful assistant. My name is Marketer John.\
        You help write the body of an email for a fictitious company called "Awesome Web Analytics". \
        This is a web analytics company that is similar to the top 5 web analytics companies \
        (Perform a web search to determine current top 5 web analytics companies). \
        The goal is to write a custom email to users to get them to be interested in our services. \
        The email should be less than 150 words. Address the user by name. End with my signature.'

## Loop through and request

Now we'll loop through each person and generate the emails by querying ChatGPT. Storing the responses in a python list.

**Note:** for the ten people we have listed above, the execution of this step takes about **1m**.

In [ ]:

def chatgpt_generate_email(prompt, person):
    conversation = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": person},
        {"role": "assistant", "content": ""}
    ]

    # Call the OpenAI Chat API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=conversation,
        temperature=0.7,
        max_tokens=800,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    assistant_reply = response.choices[0].message['content']
    return assistant_reply

In [ ]:
# Define a list to store the responses
responses = []

# Loop through each person and generate the conversation
for person in people:
    # Generate the email
    reply = chatgpt_generate_email(system, person)
    # Store the response in the dictionary
    responses.append({"person": person, "assistant_reply": reply})

## Print out the list

Now we print out the emails, with names and everything.

In [ ]:
# Print the responses
for response in responses:
    person = response["person"]
    assistant_reply = response["assistant_reply"]
    print(f"Person: {person}")
    print(f"{assistant_reply}")
    print("_"*100)

# Customizing the content to user behavior

Leveling up your game can be done pretty easily, if you have access to customer behavior data inside your app. For instance, what if a user signed up for your service but hadn't finished installing the analytics tracking code on their website?

We have a Mongo Database populated with some fake user data that looks something like this:

``` json
{
    '_id': ObjectId('64afb3fda9295d8421e7a19f'), 
    'first_name': 'James', 
    'last_name': 'Villanueva', 
    'company_name': 'Foley-Turner', 
    'stage': 'generating a tracking code', 
    'created_date': datetime.datetime(2023, 6, 26, 0, 0)
}
```

Take particular note of the key `stage`, we will be using that to help identify user behavior and have ChatGPT enhance our emails.

Below we use MongoDB as a source for user behavior, then use that to have ChatGPT write some more useful emails for the users.

In [ ]:
!pip3 install pymongo --quiet

from pymongo import MongoClient

client = MongoClient("")
mongo_db = client['webinar-mktg-email-demo']
collection = mongo_db['customers']

stages = [
    "getting started",
    "generating a tracking code",
    "adding tracking to your website",
    "real-time analytics",
    "conversion tracking",
    "funnels",
    "user segmentation",
    "custom event tracking",
    "data export",
    "dashboard customization"
]

def find_next_stage(stage):
    index = stages.index(stage)
    return stages[index+1]

## Loop through users in collection

Limiting to 10 users for the demo, we will loop through each record in our customer collection in Mongo.

In [ ]:
limit = 10
count = 0
emails = []

for record in collection.find():
    if count != limit:
        fname = record['first_name']
        stage = record['stage']

        next_stage = find_next_stage(stage)
        system = 'You are a helpful assistant, who works for me, Marketer John at Awesome Web Analytics. \
            You help write the body of an email for a fictitious company called "Awesome Web Analytics". \
            We are a web analytics company that is similar to the top 5 web analytics companies \
            (Perform a web search to determine current top 5 web analytics companies). \
            We have users that are at various stages of the pipeline of using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Please write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure that the email describes the benefits of moving to the next stage. \
            Limit the email to 1 paragraph. End email with my signature.'.format(fname, stage, next_stage)
        
        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage, "email": reply})
        
        count += 1

Now let's take a look at what those emails look like now:

In [ ]:
# Print the responses
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

# What about adding in vectors?

It'd be useful to be able to drive users to your documentation to ensure that they are able to complete what they're working on. So let's use a Vector Database (Pinecone) to store documentation, then perform a query against it to determine the best place in our documentation to send our users.

## Setup Pinecone Client

In [ ]:
# FIXME: Move to a pre-requisites section to run before the webinar
from tqdm.autonotebook import tqdm
!pip install pinecone-client --quiet

pinecone_api_key = ""

import pinecone      

pinecone.init(      
	api_key=pinecone_api_key,      
	environment='us-west4-gcp-free'      
)

pine_index = pinecone.Index('web-app-docs')

## Create Embeddings for our Stages

We need embeddings for each of the stages in our pipeline, so we can perform searches in Pinecone for the best docs.

In [ ]:
model_id = 'text-embedding-ada-002'

def getEmbeddings(text, model_id):
    try:
            text = text.replace("\n", " ")
            response = openai.Embedding.create(input=text,model=model_id)

            embedding = response['data'][0]['embedding']
            tokens = response['usage']['total_tokens']
            status = 'success'
            return embedding,tokens,status
    except Exception as e:
        print(e)
        embedding = ''
        tokens = 0
        status = 'failed'
        return embedding,tokens,status

In [ ]:
stages_w_embed = []

for s in stages:
    embedding,tokens,status = getEmbeddings(s, model_id)
    stages_w_embed.append({"stage": s, "embedding": embedding})

## Search Pinecone for Closest Result

Using the embeddings for each stage, generated in the last step, we can search the Pinecone index for the closest matching document in our docs.

**Note:** This will take about **1m** to run.

In [ ]:
def search_pinecone(embedding):
    response = pine_index.query(queries=[embedding], top_k=1, namespace='docs', include_metadata=True)
    metadata = response['results'][0]['matches'][0]['metadata']
    content = metadata['content']
    permalink = metadata['parent']
    return content, permalink

In [ ]:
limit = 10
count = 0
emails = []

for record in collection.find():
    if count != limit:
        fname = record['first_name']
        stage = record['stage']

        next_stage = find_next_stage(stage)
        this_stage = next((item for item in stages_w_embed if item['stage'] == stage), None)
        next_stage = next((item for item in stages_w_embed if item['stage'] == next_stage), None)

        cur_content, cur_permalink = search_pinecone(this_stage['embedding'])
        next_content, next_permalink = search_pinecone(next_stage['embedding'])

        cur_link = f'https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{cur_permalink}.md'
        next_link = f'https://github.com/singlestore-labs/webinar-code-examples/blob/main/mktg-email-flow/docs/{next_permalink}.md'

        system = 'You are a helpful assistant. I am Marketer John at Awesome Web Analytics. \
            We are similar to the current top web analytics companies \
            We have users that are at various stages in using our product and we want to \
            send them helpful emails to get them to use our product more. \
            Write an email for {} who is on stage {} of the on-boarding process. The next stage is {}. \
            Ensure that the email describes the benefits of moving to the next stage. \
            Limit the email to 1 paragraph. \
            Every email must include links to the content on our documentation page which are as follows: \
            Next Stage: {} \
            End email with my signature "Best Regards, \n Marketer John.'.format(fname, stage, next_stage, cur_link, next_link)

        reply = chatgpt_generate_email(system, person)
        emails.append({"fname": fname, "stage": stage, "next_stage": next_stage['stage'], "email": reply})
        
        count += 1

Print out the emails

In [ ]:
for e in emails:
    print(f"First Name: {e['fname']}\n")
    print(f"Stage: {e['stage']}\n")
    print(f"Next Stage: {e['next_stage']}\n")
    print(f"{e['email']}\n")
    print("_"*100)

# Lets make this easier with SingleStoreDB

In [ ]:
# Code stuff here

# Conclusion

Write conclusion here.